In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import urllib
import requests
import json

# Users

In [2]:
def get_locale(x):
    if x.find('_') != -1:
        return x.split('_')[1]
    else:
        return np.nan

def get_birth_year(x):
    if x.isdigit():
        return x
    else:
        return np.nan
    
def get_joined_at(x):
    if type(x) is str and x.find('T') != -1:
        return x.split('T')[0]
    else:
        return np.nan
    
users = pd.read_csv('data/users.csv')
users.set_index('user_id', inplace=True)

users.locale = users.locale.apply(get_locale)
users.birthyear = users.birthyear.apply(get_birth_year)
users.joinedAt = users.joinedAt.apply(get_joined_at)

users.head()

,locale,birthyear,gender,joinedAt,location,timezone
user_id,,,,,,
3197468391,ID,1993,male,2012-10-02,Medan Indonesia,480.0
3537982273,ID,1992,male,2012-09-29,Medan Indonesia,420.0
823183725,US,1975,male,2012-10-06,Stratford Ontario,-240.0
1872223848,US,1991,female,2012-11-04,Tehran Iran,210.0
3429017717,ID,1995,female,2012-09-10,NaN,420.0


In [3]:
googleGeocodeUrl = 'https://maps.googleapis.com/maps/api/geocode/json?'

def geocode(address):
    params = {
        'address': address}

    url = googleGeocodeUrl + urllib.parse.urlencode(params)
    print(url)
    response = urllib.request.urlopen(url)
    response_list = list(response)
    line = json.loads(''.join([x.decode("utf-8") for x in response_list]))
    latitude = line['results'][0]['geometry']['location']['lat']
    longitude = line['results'][0]['geometry']['location']['lng']
    return latitude, longitude


locations = pd.DataFrame({'location': users.location, 'count': 1})
location_counts = locations.groupby('location').count()

lats, lngs = [], []
for location in location_counts.index:
    try:
        lat, lng = geocode(location)
        lats.append(lat)
        lngs.append(lng)
    except:
        print('Error location: %s' % location)
        lats.append(np.nan)
        lngs.append(np.nan)

location_counts['lat'] = lats
location_counts['lng'] = lngs

https://maps.googleapis.com/maps/api/geocode/json?address=++
Error location:   
https://maps.googleapis.com/maps/api/geocode/json?address=Abadan++Iran
https://maps.googleapis.com/maps/api/geocode/json?address=Abadeh
https://maps.googleapis.com/maps/api/geocode/json?address=Abbottabad
https://maps.googleapis.com/maps/api/geocode/json?address=Aberdeen
https://maps.googleapis.com/maps/api/geocode/json?address=Aberdeen++SD
https://maps.googleapis.com/maps/api/geocode/json?address=Abhar
https://maps.googleapis.com/maps/api/geocode/json?address=Abia++Ebonyi++Nigeria
https://maps.googleapis.com/maps/api/geocode/json?address=Abidjan++82
https://maps.googleapis.com/maps/api/geocode/json?address=Abidjan++Cote+d%27Ivoire
https://maps.googleapis.com/maps/api/geocode/json?address=Ablagh++05
Error location: Ablagh  05
https://maps.googleapis.com/maps/api/geocode/json?address=Absecon++NJ
https://maps.googleapis.com/maps/api/geocode/json?address=Abu+Dhabi++01
https://maps.googleapis.com/maps/api/geoco

In [4]:
users = users.join(location_counts, on=['location'])
users.to_csv('data/superset/users.csv')
users.head()

# Users friends

In [11]:
user_friends = pd.read_csv('data/user_friends.csv')
user_friends.set_index('user', inplace=True)

user_friends.friends = user_friends.friends.apply(lambda x: list(map(int, x.split())) if x is not np.nan else [])

user_friends.head()

,friends
user,
3197468391,"[1346449342, 3873244116, 4226080662, 122290762..."
3537982273,"[1491560444, 395798035, 2036380346, 899375619,..."
823183725,"[1484954627, 1950387873, 1652977611, 418596082..."
1872223848,"[83361640, 723814682, 557944478, 1724049724, 2..."
3429017717,"[4253303705, 2130310957, 1838389374, 392873576..."


In [12]:
users, friends = [], []
for user, row in user_friends.iterrows():
    for friend in row['friends']:
        users.append(user)
        friends.append(friend)

user_friends = pd.DataFrame({'user_id': users, 'friend_id': friends})
user_friends.set_index('user_id', inplace=True)
user_friends.to_csv('data/superset/user_friends.csv')

# Events

In [14]:
def get_start_time(x):
    if type(x) is str and x.find('T') != -1:
        return x.split('T')[0]
    else:
        return np.nan

events = pd.read_csv('data/events.csv')
events.set_index('event_id', inplace=True)

events.start_time = events.start_time.apply(get_start_time)

events.head()

,user_id,start_time,city,state,zip,country,lat,lng,c_1,c_2,...,c_92,c_93,c_94,c_95,c_96,c_97,c_98,c_99,c_100,c_other
event_id,,,,,,,,,,,,,,,,,,,,,
684921758,3647864012,2012-10-31,NaN,NaN,NaN,NaN,NaN,NaN,2,0,...,0,1,0,0,0,0,0,0,0,9
244999119,3476440521,2012-11-03,NaN,NaN,NaN,NaN,NaN,NaN,2,0,...,0,0,0,0,0,0,0,0,0,7
3928440935,517514445,2012-11-05,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,12
2582345152,781585781,2012-10-30,NaN,NaN,NaN,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,0,0,8
1051165850,1016098580,2012-09-27,NaN,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,0,0,0,9


# Event attendees

In [47]:
event_attendees = pd.read_csv('data/event_attendees.csv', index_col='event')

event_attendees.yes = event_attendees.yes.apply(lambda x: map(int, x.split()) 
                                                if x is not np.nan else np.nan)
event_attendees.maybe = event_attendees.maybe.apply(lambda x: map(int, x.split()) 
                                                    if x is not np.nan else np.nan)
event_attendees.no = event_attendees.no.apply(lambda x: map(int, x.split()) 
                                              if x is not np.nan else np.nan)
event_attendees.invited = event_attendees.invited.apply(lambda x: map(int, x.split()) 
                                                        if x is not np.nan else np.nan)

event_attendees.head()

,yes,maybe,invited,no
event,,,,
1159822043,"[1975964455, 252302513, 4226086795, 3805886383...","[2733420590, 517546982, 1350834692, 532087573,...","[1723091036, 3795873583, 4109144917, 356062290...","[3575574655, 1077296663]"
686467261,"[2394228942, 2686116898, 1056558062, 379294223...","[1498184352, 645689144, 3770076778, 331335845,...","[1788073374, 733302094, 1830571649, 676508092,...",NaN
1186208412,NaN,"[3320380166, 3810793697]","[1379121209, 440668682]","[1728988561, 2950720854]"
2621578336,NaN,NaN,NaN,NaN
855842686,"[2406118796, 3550897984, 294255260, 1125817077...","[2671721559, 1761448345, 2356975806, 266666946...","[1518670705, 880919237, 2326414227, 2673818347...",[3500235232]


In [74]:
event_counts = deepcopy(event_attendees)
event_counts.yes = event_counts.yes.apply(lambda x: len(x) if x is not np.nan else 0)
event_counts.maybe = event_counts.maybe.apply(lambda x: len(x) if x is not np.nan else 0)
event_counts.no = event_counts.no.apply(lambda x: len(x) if x is not np.nan else 0)
event_counts.invited = event_counts.invited.apply(lambda x: len(x) if x is not np.nan else 0)
event_counts['count'] = 1

# Train

In [19]:
train = pd.read_csv('data/train.csv')
train.head()

,user,event,invited,timestamp,interested,not_interested
0,3044012,1918771225,0,2012-10-02 15:53:05.754000+00:00,0,0
1,3044012,1502284248,0,2012-10-02 15:53:05.754000+00:00,0,0
2,3044012,2529072432,0,2012-10-02 15:53:05.754000+00:00,1,0
3,3044012,3072478280,0,2012-10-02 15:53:05.754000+00:00,0,0
4,3044012,1390707377,0,2012-10-02 15:53:05.754000+00:00,0,0


# Test

In [101]:
test = pd.read_csv('data/test.csv')
test.head()

,user,event,invited,timestamp
0,1776192,2877501688,0,2012-11-30 11:39:01.230000+00:00
1,1776192,3025444328,0,2012-11-30 11:39:01.230000+00:00
2,1776192,4078218285,0,2012-11-30 11:39:01.230000+00:00
3,1776192,1024025121,0,2012-11-30 11:39:01.230000+00:00
4,1776192,2972428928,0,2012-11-30 11:39:21.985000+00:00


In [2]:
clickhouse_url = 'http://default@localhost:8123/?'
clickhouse_query_url = 'http://localhost:8123/?query="%s"'

def create_tables(filename, recreate=True):
    if recreate == True:
        r = requests.post(clickhouse_url, 
                      data=open('data/superset/drop_%s.sql' % filename, 'rb').read(),
                      headers={'Content-Type': 'application/octet-stream'})
        print(r.reason)
    r = requests.post(clickhouse_url, 
                      data=open('data/superset/create_%s.sql' % filename, 'rb').read(),
                      headers={'Content-Type': 'application/octet-stream'})
    print(r.reason)

    
def insert_values_from_csv(tablename):
    r = requests.post(clickhouse_url + 'query=INSERT INTO mydb.users FORMAT CSVWithNames',
                      data=open('data/superset/%s.csv' % tablename, 'rb'),
                      headers={'content-type': 'text/csv'})

    print(r.reason)
    print(r.text)

In [18]:
create_tables('users')
insert_values_from_csv('users')

OK
OK
OK



In [ ]:
create_tables('user_friends')
insert_values_from_csv('user_friends')

OK
OK
